In [1]:
import pymupdf
import logging
import glob
import datetime
import os
from pathlib import Path
from pymupdf import Point, Matrix, Rect, Quad

class MyRect(Rect):
    def expand_rect(self, pix=18):
        return MyRect(self[0] - pix, self[1] - pix, self[2] + pix, self[3] + pix)
    
    def center(self):
        return Point((self[0] + self[2]) / 2, (self[1] + self[3]) / 2)

    def fits_on_printer_pater(self):
        return ((min(self.width, self.height) < 7.5 * 72) and (max(self.width, self.height) < 10 * 72))
    
def merge_overlapping_rects(list_of_rects):
    final_rects = [list_of_rects.pop()]

    while len(list_of_rects) > 0:
        current_rect = list_of_rects.pop()
        sorted = False
        for final_rect in final_rects:
            if final_rect.intersects(current_rect):
                final_rect = final_rect.include_rect(current_rect)
                sorted = True
        if not sorted:
            final_rects.append(current_rect)
    
    return final_rects

In [2]:
cwd = Path.cwd()

file_path = list(cwd.glob("*[!test].pdf"))[-1]

doc = pymupdf.open(file_path)
file_path

WindowsPath('c:/Users/BenjaminAnderson/OneDrive - db Engineering/Documents/Matt task - gathering screenshots/SAMPLE DDSet_Electrical.pdf')

In [12]:
total_doc_screenshots = []

# for page_number in range(doc.page_count):
for page_number in [3,4,5,6,7,8,9]:
    page = doc[page_number]
    annots = list(page.annots())
    print(page.number)
    print(total_doc_screenshots)

    # for annot in annots:
    if len(annots) > 0:
        print(f"pg. {page.number + 1} - {annots[0].info['content']=}")
        
        # page.get_pixmap().pil_image()
        rect_groups = [[annots[0]]]

        # Group together annotations that are close to each other
        for i in range(1,len(annots)):
            annot = annots[i]
            added_to_group = False
            annot_rect = MyRect(annot.apn_bbox)

            for j in range(len(rect_groups)):
                group = rect_groups[j]
                for k in range(len(group)):
                    rect = MyRect(group[k].apn_bbox)

                    if annot_rect.expand_rect().intersects(rect.expand_rect()):
                        group.append(annot)
                        k = len(group) + 1
                        j = len(rect_groups) + 10
            
            if j != len(rect_groups) + 10:
                rect_groups.append([annot])

        # Make boundary rects that include all annots within their boundaries
        boundary_rects = []
        for rect_group in rect_groups:
            surrounding_rect = MyRect(rect_group[0].apn_bbox)

            if len(rect_group) > 0:
                for i in range(1, len(rect_group)):
                    surrounding_rect = surrounding_rect.include_rect(rect_group[i].apn_bbox)
                
            boundary_rects.append(surrounding_rect.expand_rect())

        # Make a really immature grouping of annotations
        screenshot_rects = boundary_rects.copy()

        for i in range(30):
            for j in range(len(screenshot_rects)):
                if screenshot_rects[j].fits_on_printer_pater():
                    screenshot_rects[j] = screenshot_rects[j].expand_rect(pix=8)
            
            screenshot_rects = merge_overlapping_rects(screenshot_rects)
        print(total_doc_screenshots)

        final_number = len(screenshot_rects)

        screenshot_rects = boundary_rects.copy()
        while len(screenshot_rects) > final_number:
            
            for j in range(len(screenshot_rects)):
                if screenshot_rects[j].fits_on_printer_pater():
                    screenshot_rects[j] = screenshot_rects[j].expand_rect(pix=3).intersect(page.rect)
            
            screenshot_rects = merge_overlapping_rects(screenshot_rects)
        print(total_doc_screenshots)

        # Make a dictionary containing all the annotations and key information for this page
        print(screenshot_rects)
        screenshot_dict = {rect: {"annot_ids": [], "authors": set(), "last_modified": -1, "portrait": True} for rect in screenshot_rects}
        print(total_doc_screenshots)

        for annot in annots:
            for screenshot_rect in screenshot_dict.keys():
                if annot.apn_bbox.intersects(screenshot_rect):
                    screenshot_dict[screenshot_rect]["annot_ids"].append(annot.info['id'])
                    screenshot_dict[screenshot_rect]["authors"].add(annot.info['title'])

                    modif_time = datetime.datetime.strptime(annot.info['modDate'].replace("'", ""), r"D:%Y%m%d%H%M%S%z")

                    if screenshot_dict[screenshot_rect]["last_modified"] == -1 or modif_time.timestamp() > screenshot_dict[screenshot_rect]["last_modified"].timestamp():
                        screenshot_dict[screenshot_rect]["last_modified"] = modif_time

                    if screenshot_rect.width > screenshot_rect.height:
                        screenshot_dict[screenshot_rect]["portrait"] = False

        total_doc_screenshots.append((page.number, page.get_label(), screenshot_dict))
        # print(total_doc_screenshots)

3
[]
4
[]
5
[]
6
[]
7
[]
8
[]
9
[]
pg. 10 - annots[0].info['content']=''
[]
[]
[Rect(1094.183349609375, 1360.94921875, 1306.5196533203125, 1486.14208984375)]
[]


In [22]:
annots[1].info
annots[0].colors

{'stroke': [0.501960813999176, 1.0, 0.0], 'fill': []}

# Writing output

In [14]:
page_size = pymupdf.paper_sizes()['letter']

In [20]:
output = pymupdf.open()

for page_no,page_label,screenshot_dict in total_doc_screenshots:

    image_number = 0
    for screenshot in screenshot_dict.keys():
        image_number += 1

        # screenshot_dict[screenshot]["authors"]
        # screenshot_dict[screenshot]["last_modified"]
        # screenshot_dict[screenshot]["portrait"]

        if screenshot_dict[screenshot]["portrait"]:
            new_page = output.new_page(width=page_size[0], height=page_size[1])
        else:
            new_page = output.new_page(width=page_size[1], height=page_size[0])

        new_page.insert_image(Rect((36,36),screenshot.width, screenshot.height), pixmap = doc[page_no].get_pixmap(clip=screenshot,dpi=72))

        r = pymupdf.Rect(30, min(screenshot.height + 10, new_page.mediabox[3] - 106), 210, 80)

        # new_page.draw_rect(r, color=[1,1,1], fill=[1,1,1])

        new_page.insert_text((36, min(screenshot.height + 20, new_page.mediabox[3] - 96)), f"{page_label}, image {image_number} of {len(screenshot_dict.keys())}")
        new_page.insert_text((36, min(screenshot.height + 40, new_page.mediabox[3] - 76)), f"Annotation author: {", ".join(screenshot_dict[screenshot]["authors"])}")
        new_page.insert_text((36, min(screenshot.height + 60, new_page.mediabox[3] - 56)), f"Annotation date: {screenshot_dict[screenshot]["last_modified"].strftime("%Y-%m-%d")}")

In [21]:
# screenshot_dict[screenshot]["authors"]

In [22]:
output.save(file_path.name.split(".pdf")[0] + "_summary_test.pdf")